In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.nn.utils as utils

from torch import Tensor
from torch.utils.data import Dataset, DataLoader
from torchvision import models
from torchvision import transforms, utils
from torchvision.io import read_image, write_png

import random
import os
import cv2
import zipfile
import math
import time
import argparse
import datetime
import sklearn
import pandas as pd
import numpy as np

from PIL import Image
from io import BytesIO
from math import exp
from tensorboardX import SummaryWriter
from zipfile import ZipFile
from sklearn.utils import shuffle

In [2]:
class Sigm(nn.Module):
    def forward(self, input: Tensor) -> Tensor:
        return 255*torch.sigmoid(input)

In [3]:
class MyNet(nn.Module):
    def __init__(self):
        super(MyNet, self).__init__()
        self.layer1 = nn.Sequential(nn.Conv2d(in_channels=3, out_channels=8, kernel_size=3, padding=1), nn.ReLU())
        self.layer2 = nn.Sequential(nn.Conv2d(in_channels=8, out_channels=8, kernel_size=3, padding=1), nn.ReLU())
        self.p1 = nn.MaxPool2d(2)
        self.bn1 = nn.BatchNorm2d(8)
        
        self.layer3 = nn.Sequential(nn.Conv2d(in_channels=8, out_channels=8, kernel_size=3, padding=1), nn.ReLU())
        self.layer4 = nn.Sequential(nn.Conv2d(in_channels=8, out_channels=8, kernel_size=3, padding=1), nn.ReLU())
        self.p2 = nn.MaxPool2d(2)
        self.bn2 = nn.BatchNorm2d(8)
        
        self.layer5 = nn.Sequential(nn.Conv2d(in_channels=8, out_channels=32, kernel_size=3, padding=1), nn.ReLU())
        self.layer6 = nn.Sequential(nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, padding=1), nn.ReLU())
        self.p3 = nn.MaxPool2d(2)
        self.bn3 = nn.BatchNorm2d(64)
        
        self.layer7 = nn.Sequential(nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3, padding=1), nn.ReLU())
        self.layer8 = nn.Sequential(nn.Conv2d(in_channels=128, out_channels=256, kernel_size=3, padding=1), nn.ReLU())
        self.layer9 = nn.Sequential(nn.Conv2d(in_channels=256, out_channels=256, kernel_size=3, padding=1), nn.ReLU())
        self.p4 = nn.MaxPool2d(2)
        self.bn4 = nn.BatchNorm2d(256)

        self.layer10 = nn.Sequential(nn.Conv2d(in_channels=256, out_channels=512, kernel_size=3, padding=1), nn.ReLU())
        self.layer11 = nn.Sequential(nn.Conv2d(in_channels=512, out_channels=512, kernel_size=3, padding=1), nn.ReLU())
        self.layer12 = nn.Sequential(nn.Conv2d(in_channels=512, out_channels=1024, kernel_size=3, padding=1), nn.ReLU())
        self.p5 = nn.MaxPool2d(2)
        self.bn5 = nn.BatchNorm2d(1024)
        
        self.up1 = nn.Upsample(scale_factor=2, mode='bilinear')
        self.layer13 = nn.Sequential(nn.Conv2d(in_channels=1024, out_channels=2048, kernel_size=3, padding=1), nn.ReLU())
        self.layer14 = nn.Sequential(nn.Conv2d(in_channels=2048, out_channels=2048, kernel_size=3, padding=1), nn.ReLU())
        self.layer15 = nn.Sequential(nn.Conv2d(in_channels=2048, out_channels=1024, kernel_size=3, padding=1), nn.ReLU())
        self.bn6 = nn.BatchNorm2d(1024)
        
        self.up2 = nn.Upsample(scale_factor=2, mode='bilinear')
        self.layer16 = nn.Sequential(nn.Conv2d(in_channels=1024, out_channels=1024, kernel_size=3, padding=1), nn.ReLU())
        self.layer17 = nn.Sequential(nn.Conv2d(in_channels=1024, out_channels=512, kernel_size=3, padding=1), nn.ReLU())
        self.layer18 = nn.Sequential(nn.Conv2d(in_channels=512, out_channels=256, kernel_size=3, padding=1), nn.ReLU())
        self.bn7 = nn.BatchNorm2d(256)
        
        self.up3 = nn.Upsample(scale_factor=2, mode='bilinear')
        self.layer19 = nn.Sequential(nn.Conv2d(in_channels=256, out_channels=128, kernel_size=3, padding=1), nn.ReLU())
        self.layer20 = nn.Sequential(nn.Conv2d(in_channels=128, out_channels=128, kernel_size=3, padding=1), nn.ReLU())
        self.layer21 = nn.Sequential(nn.Conv2d(in_channels=128, out_channels=64, kernel_size=3, padding=1), nn.ReLU())
        
        self.up4 = nn.Upsample(scale_factor=2, mode='bilinear')
        self.layer22 = nn.Sequential(nn.Conv2d(in_channels=64, out_channels=32, kernel_size=3, padding=1), nn.ReLU())
        self.layer23 = nn.Sequential(nn.Conv2d(in_channels=32, out_channels=16, kernel_size=3, padding=1), nn.ReLU())
  
        self.up5 = nn.Upsample(scale_factor=2, mode='bilinear')
        self.layer24 = nn.Sequential(nn.Conv2d(in_channels=16, out_channels=1, kernel_size=3, padding=1), Sigm())
       
    def forward(self, features):
        out = self.layer1(features)
        out = self.layer2(out)
        out = self.p1(out)
        out = self.bn1(out)
        
        out = self.layer3(out)
        out = self.layer4(out)
        out = self.p2(out)
        out = self.bn2(out)
        
        out = self.layer5(out)
        out = self.layer6(out)
        out = self.p3(out)
        out = self.bn3(out)
        
    
        out = self.layer7(out)
        out = self.layer8(out)
        out = self.layer9(out)
        out = self.p4(out)
        out = self.bn4(out)
        
        out = self.layer10(out)
        out = self.layer11(out)
        out = self.layer12(out)
        out = self.p5(out)
        out = self.bn5(out)

        out = self.up1(out)
        out = self.layer13(out)
        out = self.layer14(out)
        out = self.layer15(out)
        out = self.bn6(out)
        
        out = self.up2(out)
        out = self.layer16(out)
        out = self.layer17(out)
        out = self.layer18(out)
        out = self.bn7(out)
        
        out = self.up3(out)
        out = self.layer19(out)
        out = self.layer20(out)
        out = self.layer21(out)
        
        out = self.up4(out)
        out = self.layer22(out)
        out = self.layer23(out)

        out = self.up5(out)
        out = self.layer24(out)
        
        return out


In [11]:
img = torch.zeros(1, 3, 640, 480).cuda()

In [4]:
def get_img_data(filename):
  return cv2.imread(filename)

In [4]:
def loadZipToMem(zip_file):
    # Load zip file into memory
    print('Loading dataset zip file...', end='')
    input_zip = ZipFile(zip_file)
    data = {name: input_zip.read(name) for name in input_zip.namelist()}

    nyu2_train = list((row.split(',') for row in (data['data/nyu2_train.csv']).decode("utf-8").split('\n') if len(row) > 0))
    nyu2_train = shuffle(nyu2_train, random_state=0)

    print('Loaded ({0}).'.format(len(nyu2_train)))
    return data, nyu2_train

class depthDatasetMemory(Dataset):
    def __init__(self, nyu2_train):
        self.nyu_dataset = nyu2_train

    def __getitem__(self, idx):
        sample = self.nyu_dataset[idx]
        image = read_image(sample[0]).type(torch.float32) 
        depth = read_image(sample[1]).type(torch.float32) 

        return image, depth

    def __len__(self):
        return len(self.nyu_dataset)



In [18]:
# fantasy_zip = zipfile.ZipFile('../nyu_data.zip')
# fantasy_zip.extractall()
# fantasy_zip.close()

In [5]:
class AverageMeter(object):
    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

def DepthNorm(depth, maxDepth=255.0): 
    return maxDepth / depth


In [6]:
def _mask_input(input, mask=None):
    if mask is not None:
        input = input * mask
        count = torch.sum(mask).data[0]
    else:
        count = np.prod(input.size(), dtype=np.float32).item()
    return input, count
class BerHuLoss(nn.Module):
    def forward(self, input, target, mask=None):
        x = input - target
        abs_x = torch.abs(x)
        c = torch.max(abs_x).item() / 5
        leq = (abs_x <= c).float()
        l2_losses = (x ** 2 + c ** 2) / (2 * c)
        losses = leq * abs_x + (1 - leq) * l2_losses
        losses, count = _mask_input(losses, mask)
        return torch.sum(losses) / count


In [7]:
def main(model_name):
    epochs = 20
    lr = 0.0001
    batch_size = 1

    model = MyNet().cuda()
    print('Model created.')

    optimizer = torch.optim.Adam(model.parameters(), lr)
    
    data, nyu2_train = loadZipToMem('../nyu_data.zip')
    dataset = depthDatasetMemory(nyu2_train)
    train_dataset, test_dataset = torch.utils.data.random_split(dataset, [int(math.floor(len(nyu2_train) * 0.8)), int(math.ceil(len(nyu2_train) * 0.2))])

    train_loader = DataLoader(dataset=train_dataset,
                                     batch_size=batch_size,
                                     shuffle=False)

    test_loader = DataLoader(dataset=test_dataset,
                                     batch_size=batch_size,
                                     shuffle=True)

    l1_criterion = BerHuLoss()

    for epoch in range(epochs):
        batch_time = AverageMeter()
        losses = AverageMeter()
        N = len(train_loader)

        model.train()

        end = time.time()
        for i, (img, dph) in enumerate(train_loader):
            optimizer.zero_grad()

            image = torch.autograd.Variable(img.cuda())
            depth = torch.autograd.Variable(dph.cuda(non_blocking=True))
            
            output = model(image)

            loss = l1_criterion(output, depth)
    
            # Update step
            losses.update(loss.data.item(), image.size(0))
            loss.backward()
            optimizer.step()

            # Measure elapsed time
            batch_time.update(time.time() - end)
            end = time.time()
            eta = str(datetime.timedelta(seconds=int(batch_time.val*(N - i))))
        
            # Log progress
            niter = epoch*N+i
            if i % 5 == 0:
                print('Epoch: [{0}][{1}/{2}]\t'
                'Time {batch_time.val:.3f} ({batch_time.sum:.3f})\t'
                'ETA {eta}\t'
                'Loss {loss.val:.4f} ({loss.avg:.4f})'.format(epoch, i, N, batch_time=batch_time, loss=losses, eta=eta))
                
        LogProgress(model, test_loader, epoch)

        if epoch == epochs-1:
            torch.save(model.state_dict, f'w_{model_name}.pth')
            torch.save(model, f'{model_name}.pth')
            return model
            

def LogProgress(model, test_loader, epoch):
    model.eval()
    sequential = test_loader
    image, depth = next(iter(sequential))
    image, depth = image.cuda(), depth.cuda(non_blocking=True)
    
    output = model(image)
    write_png((output[0,:,:,:]).cpu().type(torch.uint8), f"{epoch}.png")
    write_png(image[0,:,:,:].cpu().type(torch.uint8), f"{epoch}i.png")
    
    del image
    del depth
    del output



In [ ]:
model = main()

Model created.
Loading dataset zip file...Loaded (50688).
Epoch: [0][0/40550]	Time 0.177 (0.177)	ETA 1:59:52	Loss 85.3919 (85.3919)
Epoch: [0][5/40550]	Time 0.092 (0.639)	ETA 1:02:04	Loss 68.9141 (118.1197)
Epoch: [0][10/40550]	Time 0.091 (1.096)	ETA 1:01:18	Loss 48.2749 (99.0147)
Epoch: [0][15/40550]	Time 0.093 (1.553)	ETA 1:02:43	Loss 44.4587 (88.0667)
Epoch: [0][20/40550]	Time 0.091 (2.011)	ETA 1:01:42	Loss 64.8606 (85.5471)
Epoch: [0][25/40550]	Time 0.092 (2.470)	ETA 1:01:55	Loss 42.7822 (75.9341)
Epoch: [0][30/40550]	Time 0.092 (2.927)	ETA 1:01:49	Loss 26.4576 (70.2308)
Epoch: [0][35/40550]	Time 0.091 (3.383)	ETA 1:01:31	Loss 46.8440 (66.5862)
Epoch: [0][40/40550]	Time 0.092 (3.841)	ETA 1:02:23	Loss 31.1445 (65.3783)
Epoch: [0][45/40550]	Time 0.091 (4.298)	ETA 1:01:33	Loss 71.9544 (64.3263)
Epoch: [0][50/40550]	Time 0.092 (4.756)	ETA 1:01:59	Loss 35.9504 (62.3910)
Epoch: [0][55/40550]	Time 0.091 (5.214)	ETA 1:01:40	Loss 25.9279 (61.4348)
Epoch: [0][60/40550]	Time 0.092 (5.671)	ETA

Epoch: [0][535/40550]	Time 0.093 (49.365)	ETA 1:01:47	Loss 16.3224 (43.0463)
Epoch: [0][540/40550]	Time 0.092 (49.826)	ETA 1:01:29	Loss 46.4610 (43.1817)
Epoch: [0][545/40550]	Time 0.092 (50.286)	ETA 1:01:12	Loss 54.6979 (43.1411)
Epoch: [0][550/40550]	Time 0.092 (50.747)	ETA 1:01:33	Loss 19.1474 (43.1755)
Epoch: [0][555/40550]	Time 0.093 (51.209)	ETA 1:01:51	Loss 51.3231 (43.2060)
Epoch: [0][560/40550]	Time 0.092 (51.670)	ETA 1:01:33	Loss 12.1541 (43.0675)
Epoch: [0][565/40550]	Time 0.092 (52.131)	ETA 1:01:17	Loss 30.0921 (42.9606)
Epoch: [0][570/40550]	Time 0.092 (52.592)	ETA 1:01:27	Loss 30.8269 (42.9270)
Epoch: [0][575/40550]	Time 0.093 (53.053)	ETA 1:01:45	Loss 24.9748 (42.8206)
Epoch: [0][580/40550]	Time 0.092 (53.514)	ETA 1:01:33	Loss 54.3258 (42.7622)
Epoch: [0][585/40550]	Time 0.092 (53.975)	ETA 1:01:06	Loss 49.0095 (42.7737)
Epoch: [0][590/40550]	Time 0.092 (54.436)	ETA 1:01:17	Loss 20.0585 (42.6820)
Epoch: [0][595/40550]	Time 0.092 (54.897)	ETA 1:01:34	Loss 119.4541 (42.6687

Epoch: [0][1070/40550]	Time 0.093 (98.792)	ETA 1:00:59	Loss 45.9631 (40.2050)
Epoch: [0][1075/40550]	Time 0.092 (99.255)	ETA 1:00:44	Loss 58.3385 (40.2788)
Epoch: [0][1080/40550]	Time 0.092 (99.717)	ETA 1:00:44	Loss 84.2960 (40.3160)
Epoch: [0][1085/40550]	Time 0.092 (100.179)	ETA 1:00:47	Loss 42.5077 (40.3092)
Epoch: [0][1090/40550]	Time 0.094 (100.641)	ETA 1:01:53	Loss 24.3364 (40.3175)
Epoch: [0][1095/40550]	Time 0.093 (101.104)	ETA 1:00:50	Loss 28.1885 (40.3060)
Epoch: [0][1100/40550]	Time 0.092 (101.567)	ETA 1:00:43	Loss 51.7086 (40.2838)
Epoch: [0][1105/40550]	Time 0.093 (102.029)	ETA 1:00:58	Loss 25.7834 (40.2088)
Epoch: [0][1110/40550]	Time 0.091 (102.490)	ETA 0:59:49	Loss 47.5326 (40.2068)
Epoch: [0][1115/40550]	Time 0.094 (102.954)	ETA 1:01:27	Loss 30.8193 (40.1289)
Epoch: [0][1120/40550]	Time 0.093 (103.416)	ETA 1:00:51	Loss 52.6882 (40.1375)
Epoch: [0][1125/40550]	Time 0.092 (103.879)	ETA 1:00:46	Loss 20.0652 (40.0800)
Epoch: [0][1130/40550]	Time 0.092 (104.341)	ETA 1:00:42

Epoch: [0][1590/40550]	Time 0.093 (147.001)	ETA 1:00:25	Loss 62.7353 (39.6684)
Epoch: [0][1595/40550]	Time 0.093 (147.465)	ETA 1:00:03	Loss 15.3909 (39.6228)
Epoch: [0][1600/40550]	Time 0.093 (147.929)	ETA 1:00:20	Loss 29.5509 (39.5953)
Epoch: [0][1605/40550]	Time 0.093 (148.394)	ETA 1:00:34	Loss 30.8085 (39.5461)
Epoch: [0][1610/40550]	Time 0.093 (148.857)	ETA 1:00:26	Loss 27.3731 (39.5060)
Epoch: [0][1615/40550]	Time 0.093 (149.322)	ETA 1:00:22	Loss 36.2251 (39.5071)
Epoch: [0][1620/40550]	Time 0.093 (149.786)	ETA 1:00:37	Loss 70.9051 (39.5548)
Epoch: [0][1625/40550]	Time 0.092 (150.250)	ETA 0:59:51	Loss 40.0449 (39.5892)
Epoch: [0][1630/40550]	Time 0.093 (150.713)	ETA 1:00:30	Loss 67.6015 (39.6006)
Epoch: [0][1635/40550]	Time 0.092 (151.177)	ETA 0:59:27	Loss 57.3292 (39.6252)
Epoch: [0][1640/40550]	Time 0.093 (151.642)	ETA 1:00:35	Loss 54.0311 (39.5902)
Epoch: [0][1645/40550]	Time 0.093 (152.107)	ETA 1:00:31	Loss 31.7778 (39.5706)
Epoch: [0][1650/40550]	Time 0.088 (152.566)	ETA 0:56

Epoch: [0][2110/40550]	Time 0.093 (195.281)	ETA 0:59:38	Loss 34.5071 (38.8952)
Epoch: [0][2115/40550]	Time 0.093 (195.745)	ETA 0:59:36	Loss 90.6725 (38.8940)
Epoch: [0][2120/40550]	Time 0.093 (196.210)	ETA 0:59:23	Loss 13.4937 (38.8702)
Epoch: [0][2125/40550]	Time 0.093 (196.674)	ETA 0:59:26	Loss 86.0314 (38.8777)
Epoch: [0][2130/40550]	Time 0.093 (197.138)	ETA 0:59:20	Loss 19.8938 (38.8583)
Epoch: [0][2135/40550]	Time 0.094 (197.603)	ETA 0:59:58	Loss 35.0227 (38.8613)
Epoch: [0][2140/40550]	Time 0.093 (198.066)	ETA 0:59:21	Loss 57.0620 (38.8661)
Epoch: [0][2145/40550]	Time 0.093 (198.531)	ETA 0:59:41	Loss 70.1256 (38.8666)
Epoch: [0][2150/40550]	Time 0.093 (198.995)	ETA 0:59:43	Loss 10.0647 (38.8234)
Epoch: [0][2155/40550]	Time 0.093 (199.460)	ETA 0:59:41	Loss 45.7860 (38.8164)
Epoch: [0][2160/40550]	Time 0.093 (199.924)	ETA 0:59:39	Loss 125.2510 (38.8487)
Epoch: [0][2165/40550]	Time 0.093 (200.389)	ETA 0:59:42	Loss 16.1222 (38.8085)
Epoch: [0][2170/40550]	Time 0.093 (200.853)	ETA 0:5

...

Epoch: [0][25220/40550]	Time 0.095 (2337.934)	ETA 0:24:17	Loss 16.3590 (34.2952)
Epoch: [0][25225/40550]	Time 0.095 (2338.397)	ETA 0:24:16	Loss 46.3280 (34.2946)
Epoch: [0][25230/40550]	Time 0.096 (2338.861)	ETA 0:24:30	Loss 26.8303 (34.2934)
Epoch: [0][25235/40550]	Time 0.091 (2339.321)	ETA 0:23:10	Loss 53.1651 (34.2973)
Epoch: [0][25240/40550]	Time 0.093 (2339.785)	ETA 0:23:43	Loss 61.1935 (34.2972)
Epoch: [0][25245/40550]	Time 0.095 (2340.250)	ETA 0:24:15	Loss 6.8311 (34.2964)
Epoch: [0][25250/40550]	Time 0.094 (2340.713)	ETA 0:24:03	Loss 14.5272 (34.2939)
Epoch: [0][25255/40550]	Time 0.094 (2341.175)	ETA 0:23:59	Loss 9.7619 (34.2907)
Epoch: [0][25260/40550]	Time 0.091 (2341.637)	ETA 0:23:14	Loss 33.0684 (34.2898)
Epoch: [0][25265/40550]	Time 0.091 (2342.100)	ETA 0:23:15	Loss 13.9799 (34.2899)
Epoch: [0][25270/40550]	Time 0.091 (2342.563)	ETA 0:23:13	Loss 65.1365 (34.2901)
Epoch: [0][25275/40550]	Time 0.094 (2343.027)	ETA 0:24:02	Loss 18.8763 (34.2908)
Epoch: [0][25280/40550]	Time 0

Epoch: [0][25730/40550]	Time 0.089 (2385.165)	ETA 0:22:04	Loss 17.0160 (34.2296)
Epoch: [0][25735/40550]	Time 0.090 (2385.628)	ETA 0:22:12	Loss 37.6989 (34.2283)
Epoch: [0][25740/40550]	Time 0.091 (2386.090)	ETA 0:22:34	Loss 21.1019 (34.2270)
Epoch: [0][25745/40550]	Time 0.091 (2386.553)	ETA 0:22:25	Loss 15.0479 (34.2251)
Epoch: [0][25750/40550]	Time 0.095 (2387.020)	ETA 0:23:27	Loss 18.1090 (34.2245)
Epoch: [0][25755/40550]	Time 0.094 (2387.481)	ETA 0:23:17	Loss 26.5379 (34.2239)
Epoch: [0][25760/40550]	Time 0.091 (2387.942)	ETA 0:22:26	Loss 43.7449 (34.2214)
Epoch: [0][25765/40550]	Time 0.094 (2388.406)	ETA 0:23:03	Loss 17.4752 (34.2215)
Epoch: [0][25770/40550]	Time 0.096 (2388.872)	ETA 0:23:39	Loss 16.1934 (34.2202)
Epoch: [0][25775/40550]	Time 0.091 (2389.332)	ETA 0:22:27	Loss 49.8721 (34.2197)
Epoch: [0][25780/40550]	Time 0.090 (2389.794)	ETA 0:22:14	Loss 27.5560 (34.2189)
Epoch: [0][25785/40550]	Time 0.094 (2390.259)	ETA 0:23:11	Loss 55.0826 (34.2195)
Epoch: [0][25790/40550]	Time

Epoch: [0][26240/40550]	Time 0.095 (2432.386)	ETA 0:22:34	Loss 35.5000 (34.1447)
Epoch: [0][26245/40550]	Time 0.091 (2432.847)	ETA 0:21:39	Loss 24.7145 (34.1436)
Epoch: [0][26250/40550]	Time 0.090 (2433.312)	ETA 0:21:30	Loss 50.1935 (34.1433)
Epoch: [0][26255/40550]	Time 0.091 (2433.774)	ETA 0:21:38	Loss 18.5321 (34.1427)
Epoch: [0][26260/40550]	Time 0.095 (2434.238)	ETA 0:22:40	Loss 13.4511 (34.1438)
Epoch: [0][26265/40550]	Time 0.092 (2434.699)	ETA 0:21:51	Loss 18.9475 (34.1424)
Epoch: [0][26270/40550]	Time 0.093 (2435.163)	ETA 0:22:08	Loss 47.2346 (34.1432)
Epoch: [0][26275/40550]	Time 0.094 (2435.628)	ETA 0:22:28	Loss 5.9803 (34.1396)
Epoch: [0][26280/40550]	Time 0.094 (2436.090)	ETA 0:22:25	Loss 18.6386 (34.1374)
Epoch: [0][26285/40550]	Time 0.091 (2436.553)	ETA 0:21:42	Loss 22.2804 (34.1348)
Epoch: [0][26290/40550]	Time 0.093 (2437.016)	ETA 0:22:11	Loss 28.1941 (34.1371)
Epoch: [0][26295/40550]	Time 0.090 (2437.479)	ETA 0:21:21	Loss 73.1196 (34.1405)
Epoch: [0][26300/40550]	Time 

Epoch: [0][26750/40550]	Time 0.092 (2479.622)	ETA 0:21:10	Loss 41.7137 (34.0926)
Epoch: [0][26755/40550]	Time 0.090 (2480.085)	ETA 0:20:35	Loss 31.8510 (34.0939)
Epoch: [0][26760/40550]	Time 0.095 (2480.550)	ETA 0:21:53	Loss 28.3210 (34.0913)
Epoch: [0][26765/40550]	Time 0.095 (2481.013)	ETA 0:21:48	Loss 19.6453 (34.0894)
Epoch: [0][26770/40550]	Time 0.097 (2481.480)	ETA 0:22:19	Loss 16.5722 (34.0886)
Epoch: [0][26775/40550]	Time 0.092 (2481.938)	ETA 0:21:00	Loss 83.4491 (34.0913)
Epoch: [0][26780/40550]	Time 0.094 (2482.402)	ETA 0:21:29	Loss 38.1770 (34.0918)
Epoch: [0][26785/40550]	Time 0.094 (2482.865)	ETA 0:21:38	Loss 25.8177 (34.0908)
Epoch: [0][26790/40550]	Time 0.092 (2483.326)	ETA 0:21:03	Loss 22.9293 (34.0880)
Epoch: [0][26795/40550]	Time 0.095 (2483.791)	ETA 0:21:43	Loss 14.3583 (34.0885)
Epoch: [0][26800/40550]	Time 0.095 (2484.254)	ETA 0:21:40	Loss 27.0756 (34.0863)
Epoch: [0][26805/40550]	Time 0.094 (2484.717)	ETA 0:21:33	Loss 79.0877 (34.0869)
Epoch: [0][26810/40550]	Time

Epoch: [0][27260/40550]	Time 0.095 (2526.865)	ETA 0:21:08	Loss 17.4690 (34.0399)
Epoch: [0][27265/40550]	Time 0.093 (2527.327)	ETA 0:20:31	Loss 15.8506 (34.0413)
Epoch: [0][27270/40550]	Time 0.091 (2527.789)	ETA 0:20:08	Loss 15.7217 (34.0397)
Epoch: [0][27275/40550]	Time 0.092 (2528.253)	ETA 0:20:15	Loss 20.5979 (34.0398)
Epoch: [0][27280/40550]	Time 0.095 (2528.717)	ETA 0:20:56	Loss 35.8323 (34.0396)
Epoch: [0][27285/40550]	Time 0.092 (2529.180)	ETA 0:20:19	Loss 21.4216 (34.0394)
Epoch: [0][27290/40550]	Time 0.093 (2529.644)	ETA 0:20:27	Loss 36.8312 (34.0388)
Epoch: [0][27295/40550]	Time 0.096 (2530.110)	ETA 0:21:13	Loss 20.4888 (34.0387)
Epoch: [0][27300/40550]	Time 0.096 (2530.573)	ETA 0:21:12	Loss 12.2291 (34.0358)
Epoch: [0][27305/40550]	Time 0.096 (2531.038)	ETA 0:21:17	Loss 56.3989 (34.0364)
Epoch: [0][27310/40550]	Time 0.095 (2531.500)	ETA 0:20:56	Loss 33.0973 (34.0352)
Epoch: [0][27315/40550]	Time 0.092 (2531.964)	ETA 0:20:20	Loss 13.1991 (34.0331)
Epoch: [0][27320/40550]	Time

In [10]:
from torchsummary import summary

In [7]:
model = torch.load("net3.pth")
# summary(model, (3, 640, 480))

In [9]:
def color_depth_map(depth_map,  colormapName='jet'):
  """
  Create coloured depth map by using colormap  
  """
  colmap = cm.get_cmap(colormapName, 256) 

  if hasattr(colmap, "colors"):
      colors = colmap.colors[...,0:3]*255
  else:
      colors = colmap(np.arange(0,colmap.N))[...,0:3]*255
  np.savetxt('cmap.csv', colors.astype(np.uint8), fmt='%d', delimiter=',')

  lut = np.loadtxt('cmap.csv', dtype=np.uint8, delimiter=',')
  clr_dm = np.zeros((*depth_map.shape,3), dtype=np.uint8)
  for i in range(len(depth_map)):
    for j in range(len(depth_map[0])):
      for z in range(3):
        clr_dm[i][j][z] = lut[depth_map[i][j]][z]
  return clr_dm

In [18]:
def get_img_data(filename):
  return cv2.imread(filename, cv2.IMREAD_GRAYSCALE)

In [20]:
from pathlib import Path
from matplotlib import cm

p = Path("")
for x in p.rglob("*"):
    if "i" not in str(x) and "net3" not in str(x) and "c" not in  str(x) and "d" not in str(x) and "png" in str(x):
        img = get_img_data(str(x))
        clr_dm = color_depth_map(img)
        cv2.imwrite(str(x)[:-5]+"c.png", clr_dm) 


/tmp/ipykernel_24881/151790006.py:5: MatplotlibDeprecationWarning: The get_cmap function was deprecated in Matplotlib 3.7 and will be removed two minor releases later. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap(obj)`` instead.
  colmap = cm.get_cmap(colormapName, 256)


In [8]:
data, nyu2_train = loadZipToMem('../nyu_data.zip')
dataset = depthDatasetMemory(nyu2_train)
train_dataset, test_dataset = torch.utils.data.random_split(dataset, [int(math.floor(len(nyu2_train) * 0.8)), int(math.ceil(len(nyu2_train) * 0.2))])

train_loader = DataLoader(dataset=train_dataset,
                                 batch_size=1,
                                 shuffle=False)

test_loader = DataLoader(dataset=test_dataset,
                                 batch_size=1,
                                 shuffle=True)

Loading dataset zip file...Loaded (50688).


In [17]:
import time 
n = len(test_loader)
l1_criterion = nn.MSELoss()
AvgMSE = 0
AvgTime = 0
for i, (t_image, t_depth) in enumerate(test_loader):
    t_image = t_image.cuda()
    t_depth = t_depth.cuda()
    start = time.time()
    t_outputs = model(t_image)
    end = time.time()
    
    AvgMSE += l1_criterion(t_outputs, t_depth).data.item()
    AvgTime += end - start
    
print(AvgMSE/n)
print(AvgTime/n)
    

724.350799394985
0.003422507192755056


In [15]:
AvgBerhu/n

25.599475555928837

In [3]:
!nvidia-smi

Sat Jun 17 01:26:28 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 450.119.03   Driver Version: 450.119.03   CUDA Version: 11.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  GeForce RTX 208...  Off  | 00000000:01:00.0 Off |                  N/A |
|  0%   49C    P8    22W / 250W |   3657MiB / 11019MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------